# pyspark session 

In [4]:
import pandas as pd
import great_expectations as ge
from pyspark.sql import SparkSession 

In [5]:
spark=SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/15 15:15:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data=[
     {'cathegory':'A', 'ID':101, 'Price':300, 'Status':True},
     {'cathegory':'B', 'ID':102, 'Price':400, 'Status':False},
     {'cathegory':'C', 'ID':103, 'Price':500, 'Status':None},
     {'cathegory':'D', 'ID':104, 'Price':600, 'Status':True}
     ]

# pandas data frame

In [10]:
pd_df=pd.DataFrame.from_dict(data)


In [11]:
pd_df

,cathegory,ID,Price,Status
0,A,101,300,True
1,B,102,400,False
2,C,103,500,None
3,D,104,600,True


# spark data frame from pandas df 

In [13]:
sdf=spark.createDataFrame(pd_df)

In [14]:
sdf

DataFrame[cathegory: string, ID: bigint, Price: bigint, Status: boolean]

In [15]:
sdf.show()

+---------+---+-----+------+
|cathegory| ID|Price|Status|
+---------+---+-----+------+
|        A|101|  300|  true|
|        B|102|  400| false|
|        C|103|  500|  NULL|
|        D|104|  600|  true|
+---------+---+-----+------+



# great_axpectation data frame from pandas df

In [16]:
ge_pd_df=ge.from_pandas(pd_df)

In [17]:
ge_pd_df

,cathegory,ID,Price,Status
0,A,101,300,True
1,B,102,400,False
2,C,103,500,None
3,D,104,600,True


# ge object from spark data frame

In [18]:
ge_sdf=ge.dataset.SparkDFDataset(sdf)

In [19]:
ge_sdf

In [22]:
ge_sdf.expect_column_values_to_not_be_null('Status')

{
  "success": false,
  "result": {
    "element_count": 4,
    "unexpected_count": 1,
    "unexpected_percent": 25.0,
    "unexpected_percent_total": 25.0,
    "partial_unexpected_list": [
      null
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [24]:
ge_sdf.expect_column_values_to_be_between('Price', 300, 600)

{
  "success": true,
  "result": {
    "element_count": 4,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [26]:
ge_sdf.expect_column_to_exist('ID')

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [27]:
sdf.count()

4

In [28]:
ge_sdf.count()

AttributeError: 'SparkDFDataset' object has no attribute 'count'

In [29]:
ge_sdf.spark_df.count()

4